## Training softmax model on train data : For 'C^2' number of classes

* This notebook is doing the same work as previous one (2.softmax_training) with one difference that in this we ll train softmax model on c^2 number of classes

In [ ]:
# importing necessary packages
import pandas as pd
import numpy as np

### loading feateurzied data matrix from disk

In [ ]:
df = pd.read_csv('./final_df.csv')
df.head()

### adding more features

In [ ]:
length = []
for s in df['word']:
    length.append(len(s))

length_2 = np.asarray(length).reshape(-1,1)
df['length'] = length_2
df.head()

In [ ]:
hyphen = []
all_cap = []
sp_char = []
cap_dot = []
number = []
st_alpha_dot = []
e_day = []
for i in df['word']:
    if('-' in i):
        hyphen.append(1)
    else:
        hyphen.append(0)
    if(i.isupper()):
        all_cap.append(1)
    else:
        all_cap.append(0)
    if(('$' in i or ',' in i or '%' in i or '!' in i or '\'' in i or '"' in i) \
       and len(i) == 1):
        sp_char.append(1)
    else:
        sp_char.append(0)
    if(i[0].isupper() and i.endswith('.')):
        cap_dot.append(1)
    else:
        cap_dot.append(0)
    if(i[0].isdigit() and i[-1].isdigit() and ('.' in i or ',' in i)):
        number.append(1)
    else:
        number.append(0)
    if(i[0].isalpha() and '.' in i):
        st_alpha_dot.append(1)
    else:
        st_alpha_dot.append(0)
    if(i.endswith('day')):
        e_day.append(1)
    else:
        e_day.append(0)
#print('\'')

In [ ]:
df['hyphen'] = hyphen
df['all_cap'] = all_cap
df['sp_char'] = sp_char
df['cap_dot'] = cap_dot
df['number'] = number
df['st_alpha_dot'] = st_alpha_dot
df['e_day'] = e_day

In [ ]:
tags = df.tag.unique()
len(tags)

### creating c^2 number of classes : possible classes from train data

In [ ]:
bigram = []
for i in df.index.values:
    if( i == 0):
        s = 'DT'
        tag = " ".join([s, df.tag[i]])
        bigram.append(tag)
    else:
        tag = " ".join([df.tag[i-1], df.tag[i]])
        bigram.append(tag)        

In [ ]:
tags = np.asarray(bigram)

In [ ]:
df['tags'] = bigram

In [ ]:
df.head()

### Data Matrix X and target coln y

In [ ]:
data = np.array(df)

x = data[:,2:-1]
x.shape
y = df['tags']
y.shape
x.shape

### Adding context in feautres (prev and next)

In [ ]:
no_cols = x.shape[1] * 5
features = np.zeros((211730,no_cols))

b = no_cols / 5
b = int(b)

In [ ]:
# taking prev and next words as feature vectors : appending 
# the stored feature vectors of the same

for i in range(x.shape[0]):
    if(i==0):
        features[i][b * 2: b* 3] = x[i]
        features[i][b* 3:b* 4] = x[i+1]
        features[i][b* 4:b* 5] = x[i+2]
    
    elif(i == 1):
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 3:b* 4] = x[i+1]
        features[i][b* 2:b* 3] = x[i]
        features[i][b* 4:b* 5] = x[i+2]
        
    elif(i == x.shape[0]-1):
        features[i][0:b* 1] = x[i-2]
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 2:b* 3] = x[i]

    elif(i == x.shape[0]-2):
        features[i][b* 3:b* 4] = x[i+1]
        features[i][0:b* 1] = x[i-2]
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 2:b* 3] = x[i]
    
    else:
        features[i][0:b* 1] = x[i-2]
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 2:b* 3] = x[i]
        features[i][b* 3:b* 4] = x[i+1]
        features[i][b* 4:b* 5] = x[i+2]

In [ ]:
features[features.shape[0]]

In [ ]:
feature_vect = features[:-3]
feature_vect.shape

In [ ]:
counts = dict(df['tags'].value_counts())

In [ ]:
for i in counts:
    if(counts[i] < 8):
        print(i)       

### importing packages for running softmax on c^2 no of classes

In [ ]:
from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import pickle

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
Y = le.transform(y)

In [ ]:
np.unique(Y).shape

In [ ]:
softmax_2 = LogisticRegression( multi_class = 'multinomial', solver = 'sag')
softmax_2.fit(feature_vect,Y)

### storing trained softmax model 

In [ ]:
pickle.dump(softmax_2, open('softmax_dual.pkl', 'wb'))

### storing c^2 no of classes mapping : we ll use this in viterbi algorithm

In [ ]:
pair = dict()
for i in range(Y.shape[0]):
    pair[y[i]] = Y[i]

In [ ]:
len(pair)

In [ ]:
pickle.dump(pair, open('pair_bigrams.pkl', 'wb'))

In [ ]:
softmax_2.predict_proba(feature_vect[3].reshape(1,-1))

In [ ]:
pair